In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np

from qiskit import QuantumCircuit
from qiskit_algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution



# Uncertainty Model
This concerns the probability function. 

We first bound it in the interval $[low, high]$
We then choose how many qubits $n$ to use, giving us $2^n$ discrete grid points
We perform the following upload
$$
\ket{0}_n \mapsto \ket{\psi}_n =\sum_{i=0}^{2^n-1}\sqrt{p^i}\ket{i}
$$

We define the affine mapping between the bounds and the discrete values of our measurements
$$
\{0,\dots, 2^n-1\} \ni i \mapsto \frac{high -low}{2^n-1}*i + low \in [low, high]
$$

In [2]:
# number of qubits to represent the uncertainty for our probability distribution
num_uncertainty_qubits = 3

# parameters for considered random distribution
S = 2.0  # initial spot price
vol = 0.4  # volatility of 40%
r = 0.05  # annual interest rate of 4%
T = 40 / 365  # 40 days to maturity

# resulting parameters for log-normal distribution
mu = (r - 0.5 * vol**2) * T + np.log(S)
sigma = vol * np.sqrt(T)
mean = np.exp(mu + sigma**2 / 2)
variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
stddev = np.sqrt(variance)

# lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
low = np.maximum(0, mean - 3 * stddev)
high = mean + 3 * stddev

uncertainty_model = LogNormalDistribution(
    num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
)

# alternatively, this uncertainty model is trained using qGAN

In [3]:
from qiskit import QuantumCircuit, execute, Aer
my_circ = QuantumCircuit(4)
my_circ.append(uncertainty_model, range(3))
my_circ.measure_all()   
my_circ.draw()

┌───────┐ ░ ┌─┐         
   q_0: ┤0      ├─░─┤M├─────────
        │       │ ░ └╥┘┌─┐      
   q_1: ┤1 P(X) ├─░──╫─┤M├──────
        │       │ ░  ║ └╥┘┌─┐   
   q_2: ┤2      ├─░──╫──╫─┤M├───
        └───────┘ ░  ║  ║ └╥┘┌─┐
   q_3: ──────────░──╫──╫──╫─┤M├
                  ░  ║  ║  ║ └╥┘
meas: 4/═════════════╩══╩══╩══╩═
                     0  1  2  3

# Payoff Function
We have maturity price $S_T$, and strike price $K$. 
We need a comparator that flips the ancilla qubit from $\ket{0}$ to $\ket{1}$ if $S_T> K$
We have a rescaling factor $c_{approx}\in [0,1]$ and $x\in [0,1]$ and our desired probability is
$$
\sin^2\left(\frac{\pi}{2}\cdot c_{approx} \cdot (x-\frac{1}{2})+\frac{\pi}{4}\right) \approx \frac{\pi}{2}\cdot c_{approx} \cdot \left(x-\frac{1}{2}\right) + \frac{1}{2}
$$
for small $c_{approx}$ which is a first order approximation.

We can construct an operator that does the following
$$
\ket{x}\ket{0} \mapsto \ket{x} \left(\cos(a\cdot x + b)\ket{0} + \sin(a\cdot x + b)\ket{1}\right)
$$





In [4]:
# set the strike price (should be within the low and the high value of the uncertainty)
strike_price = 1.896

# set the approximation scaling for the payoff function
c_approx = 0.125

# setup piecewise linear objective fcuntion
breakpoints = [low, strike_price] 
# low is the lower bound, strike price is where our payoff function starts to increase
slopes = [0, 1]
# can be float or list of floats.
# for list of floats, the floats are the slopes of the individual linear functions

offsets = [0, 0]
# the offsets of each linear function
f_min = 0
# minimum y value
f_max = high - strike_price
# maximum y value

european_call_objective = LinearAmplitudeFunction(
    num_uncertainty_qubits,
    slopes,
    offsets,
    domain=(low, high),
    image=(f_min, f_max),
    breakpoints=breakpoints,
    rescaling_factor=c_approx,
)

# construct A operator for QAE for the payoff function by
# composing the uncertainty model and the objective
num_qubits = european_call_objective.num_qubits
european_call = QuantumCircuit(num_qubits)
european_call.append(uncertainty_model, range(num_uncertainty_qubits))
european_call.append(european_call_objective, range(num_qubits))

# draw the circuit
european_call.draw()

┌───────┐┌────┐
q_0: ┤0      ├┤0   ├
     │       ││    │
q_1: ┤1 P(X) ├┤1   ├
     │       ││    │
q_2: ┤2      ├┤2   ├
     └───────┘│    │
q_3: ─────────┤3 F ├
              │    │
q_4: ─────────┤4   ├
              │    │
q_5: ─────────┤5   ├
              │    │
q_6: ─────────┤6   ├
              └────┘

In [5]:
# evaluate exact expected value (normalized to the [0, 1] interval)
x = uncertainty_model.values
y = np.maximum(0, x - strike_price)
exact_value = np.dot(uncertainty_model.probabilities, y)
exact_delta = sum(uncertainty_model.probabilities[x >= strike_price])
print("exact expected value:\t%.4f" % exact_value)
print("exact delta value:   \t%.4f" % exact_delta)

exact expected value:	0.1623
exact delta value:   	0.8098


In [6]:
from qiskit.utils import QuantumInstance
from qiskit_aer import AerSimulator
# from ModifiedIQAE.algorithms.amplitude_estimators.mod_iae import ModifiedIterativeAmplitudeEstimation
from ModifiedIQAE.algorithms.amplitude_estimators.mod_iae_updated import ModifiedIterativeAmplitudeEstimation


epsilon = 0.01
alpha = 0.05

problem = EstimationProblem(
    state_preparation=european_call,
    objective_qubits=[3],
    post_processing=european_call_objective.post_processing,
)

# qi = QuantumInstance(backend=AerSimulator(), shots=100)
# ae = ModifiedIterativeAmplitudeEstimation(
#     epsilon_target=epsilon, alpha=alpha, quantum_instance=qi)
sampler = Sampler(run_options={"shots": 100})
ae = ModifiedIterativeAmplitudeEstimation(
    epsilon_target=epsilon, alpha=alpha, sampler=sampler)
result = ae.estimate(problem, shots=100)

{1: 0.46, 0: 0.54}
{'1': 0.46, '0': 0.54}
{0: 0.35, 1: 0.65}
{'0': 0.35, '1': 0.65}
{0: 0.29, 1: 0.71}
{'0': 0.29, '1': 0.71}
{1: 0.68, 0: 0.32}
{'1': 0.68, '0': 0.32}
{1: 0.75, 0: 0.25}
{'1': 0.75, '0': 0.25}
{0: 0.4, 1: 0.6}
{'0': 0.4, '1': 0.6}


In [7]:
print(result.num_oracle_queries)

3600


In [8]:
conf_int = (
    np.array(result.confidence_interval_processed)
)
print("Exact value:        \t%.4f" % exact_value)
print(
    "Estimated value:    \t%.4f"
    % (result.estimation_processed)
)
print("Confidence interval:\t[%.4f, %.4f]" % tuple(conf_int))

Exact value:        	0.1623
Estimated value:    	0.1606
Confidence interval:	[0.1338, 0.1874]
